# Phase II Model Test
First attempt at a Phase II model, one which creases the master schedule and assigns students to courses simultaneously.

$S$ -- Set of all students

$C$ -- Set of all courses

$T$ -- Set of all periods {1,2,3,4,7,8}

$I$ -- Set of all instructors

**Variables:**

$x_{i,j,t}$ for  $i \in S, j \in C, t \in T$ -- Binary, 1 if student $i$ assigned to course $j$ in period $t$

$c_{j,t}$ for $j \in C, t \in T$ -- Binary, 1 if course $j$ to be offered in period $t$

$t_{k,j,t}$ for $k \in I, j \in C, t \in T$ -- Binary, 1 if teacher $k$ teaching course $j$ in period $t$

** Parameters:**

$P_{i,j}$ -- Preference for student $i$ on course $j$

$S_i$ -- Seniority constant, e.g., higher for seniors

$D_{i,j}$ -- Binary, 1 if course $i$ and $j$ are in the same department, i.e., if they meet the same requirement, e.g., highschool math

$T_{i,j}$ -- Binary, 1 if teacher $i$ is teaching course $j$

$Cap_j$ -- Capacity of course $j$


** Constraints: **

$\sum_{j} x_{i,j,t} =1 \quad \forall i \in S, t \in T$ -- Says students can be assigned to exactly one course per period.

$\sum_{t \neq T} \sum_{i \in C} \sum_{j \neq i \in C} D_{i,j} x_{k,i,t} = 0 \quad \forall k \in S, T \in T$ -- Says that students are not assigned to more than 1 class in each department (could always change to more nuanced number of courses per department--how???)

$\sum_{t \in T} c_{j,t} = 1 \quad \forall j \in C$ -- Says that each course taught only once

$\sum_{i \in S} x_{i,j,t} \leq Cap_j \quad \forall j \in C, t \in T$ -- Course capacity constraint

In [2]:
from pyscipopt import Model, quicksum
import numpy as np

In [23]:
# set up some fake data
## Sets
S = [0,1,2,3,4,5] # 6 students
C = [0,1,2,3] # 4 courses
T = [0,1] # two periods

## Preferences
#     0    1    2    3  -- Courses
P = [[1,   0,   1,   0], # student 0
     [0,   1,   1,   0], # student 1
     [0,   1,   0,   1], # student 2
     [1,   0,   0,   1], # student 3
     [1,   0,   1,   0], # student 4
     [0,   1,   1,   0]] # student 5

## Capacity
Cap = [3, 3, 3, 3]

## Course Proximity
#     0    1    2    3  -- Courses
D = [[0,   0,   0,   0], # 0 
     [0,   0,   1,   0], # 1
     [0,   1,   0,   1], # 2
     [1,   0,   0,   0]]

In [19]:
# Setup model
m = Model("test")

## Note, in the below I do not have a period index on students, I couldn't figure out how to work it

In [20]:
# Add Student Variables
X = {} # Variable dictionary
for i in S:
    for j in C:
            name = "Student " + str(i) + ", in course " + str(j)
            X[i,j] = m.addVar(name, vtype = 'B')

In [21]:
# Add Student assignment constraint
for i in S:
        m.addCons(quicksum(X[i,j] for j in C) == 1)

In [22]:
# Add capacity constraint
for j in C:
    m.addCons(quicksum(X[i,j] for i in S) <= Cap[j])

In [36]:
# Add course proximity constraint
for k in S:
    m.addCons(quicksum(D[i,j]*X[k,i] for j in list( set(C) - set([i])) for i in C))

TypeError: list indices must be integers or slices, not tuple

In [79]:
# Add course proximity constraint (without quicksum)
for k in S:
    expr = 0 # reset expression (should not have impact)
    for i in C:
        small_set = list(set(C) - set([int(i)])) # C - {i} list of courses without course i
        for j in small_set:
            expr = expr + D[i][j]*X[k,i]
    m.addCons(expr == 0)

In [59]:
D[1][2]

1

In [69]:
expr = X[1,2] + X[2,3]

In [40]:
expr = expr + X[2,2]

In [42]:
expr = expr - X[1,1]

In [51]:
expr

In [49]:
expr = None

In [34]:
list(set(C) - set([i]))

[1, 2, 3]

In [66]:
import pyscipopt

In [67]:
pyscipopt.LinExpr()

AttributeError: module 'pyscipopt' has no attribute 'LinExpr'

In [70]:
expr

Expr({Term(Student 1, in course 2): 1.0, Term(Student 2, in course 3): 1.0})

In [73]:
new pyscipopt.Expr()

SyntaxError: invalid syntax (<ipython-input-73-f67df4745e89>, line 1)

In [76]:
expr = 0

In [77]:
expr = expr + X[1,1]

In [78]:
expr

Expr({Term(Student 1, in course 1): 1.0, Term(): 0.0})